In [2]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [11]:
switzerland <- read_csv("heart_disease_data/processed.switzerland.data",
                        col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                                      "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"))|>
mutate(location = "Switzerland")


#  16 fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)

hungarian <- read_csv("heart_disease_data/processed.hungarian.data",
                       col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                                      "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")) |>
mutate(location = "Hungary")

cleveland <- read_csv("heart_disease_data/processed.cleveland.data",
                      col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                                      "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")) |>
mutate(location = "Cleveland")

va <- read_csv("heart_disease_data/processed.va.data",
               col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                                      "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")) |>
mutate(location = "VA Long Beach")


head(switzerland)
head(switzerland)
head(cleveland)
head(va)

heart_disease_data <- semi_join(switzerland, hungarian, by= c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                                      "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num", "location"))

heart_disease_data                             


Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal
dbl (5): age, sex, cp, chol, num

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 294 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, chol, fbs, restecg, thalach, exang, slope, ca, thal
dbl (5): age, sex, cp, oldpeak, num

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retr

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,location
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
32,1,1,95,0,?,0,127,0,.7,1,?,?,1,Switzerland
34,1,4,115,0,?,?,154,0,.2,1,?,?,1,Switzerland
35,1,4,?,0,?,0,130,1,?,?,?,7,3,Switzerland
36,1,4,110,0,?,0,125,1,1,2,?,6,1,Switzerland
38,0,4,105,0,?,0,166,0,2.8,1,?,?,2,Switzerland
38,0,4,110,0,0,0,156,0,0,2,?,3,1,Switzerland


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,location
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
32,1,1,95,0,?,0,127,0,.7,1,?,?,1,Switzerland
34,1,4,115,0,?,?,154,0,.2,1,?,?,1,Switzerland
35,1,4,?,0,?,0,130,1,?,?,?,7,3,Switzerland
36,1,4,110,0,?,0,125,1,1,2,?,6,1,Switzerland
38,0,4,105,0,?,0,166,0,2.8,1,?,?,2,Switzerland
38,0,4,110,0,0,0,156,0,0,2,?,3,1,Switzerland


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,location
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0,Cleveland
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2,Cleveland
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1,Cleveland
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0,Cleveland
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0,Cleveland
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0,Cleveland


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,location
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
63,1,4,140,260,0,1,112,1,3,2,?,?,2,VA Long Beach
44,1,4,130,209,0,1,127,0,0,?,?,?,0,VA Long Beach
60,1,4,132,218,0,1,140,1,1.5,3,?,?,2,VA Long Beach
55,1,4,142,228,0,1,149,1,2.5,1,?,?,1,VA Long Beach
66,1,3,110,213,1,2,99,1,1.3,2,?,?,0,VA Long Beach
66,1,3,120,0,0,1,120,0,-0.5,1,?,?,0,VA Long Beach


ERROR: [1m[33mError[39m in `semi_join()`:[22m
[1m[22m[33m![39m Can't join `x$chol` with `y$chol` due to incompatible types.
[36mℹ[39m `x$chol` is a <double>.
[36mℹ[39m `y$chol` is a <character>.


      3 age: age in years
      4 sex: sex (1 = male; 0 = female)
      9 cp: chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
     10 trestbps: resting blood pressure (in mm Hg on admission to the 
        hospital)
     12 chol: serum cholestoral in mg/dl
     16 fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
     19 restecg: resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria
     32 thalach: maximum heart rate achieved
     38 exang: exercise induced angina (1 = yes; 0 = no)
     40 oldpeak = ST depression induced by exercise relative to rest
     41 slope: the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
     44 ca: number of major vessels (0-3) colored by flourosopy
     51 thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
     58 num: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
        (in any major vessel: attributes 59 through 68 are vessels)

9. Missing Attribute Values: Several.  Distinguished with value -9.0.

In [ ]:
#switzerland$sex_changed <- ifelse(switzerland$sex == 1, "male", "female")
#switzerland